In [49]:
import numpy as np
import pandas as pd
import csv
#from EquityClasses import EquityShare
from datetime import datetime
from datetime import datetime as dt, timedelta
from datetime import date
from enum import IntEnum
from dataclasses import dataclass
from dateutil.relativedelta import relativedelta
from typing import List, Dict, Any

Frequency class represents the frequency of dividend payments. Assumed to be constant

In [50]:
class Frequency(IntEnum):
    ANNUAL = 1
    BIANNUAL = 2
    TRIANNUAL = 3
    QUARTERLY = 4
    MONTHLY = 12

EquityShare class represents a single equity position on the balance sheet at the modelling date.

In [51]:
@dataclass
class EquityShare:
    asset_id: int
    nace: str
    issuer: str
    issue_date: date
    dividend_yield: float
    frequency: Frequency
    market_price: float
    growth_rate: float

    @property
    def dividend_amount(self, market_value: float) -> float:
        return market_value * self.dividend_yield # Probably needs to be removed
    
    def terminal_amount(self, market_value: float, growth_rate: float, terminal_rate: float) -> float:
        return market_value/(terminal_rate-growth_rate)
    
    def generate_dividend_dates(self, modelling_date: date, end_date: date) -> date:
        """
        Generator yielding the dividend payment date starting from the first dividend
        paid after the modelling date. 
        
        :type modelling_date: date
        :type end_date: date
        
        """
        delta = relativedelta(months=(12 // self.frequency))
        this_date = self.issue_date - delta
        while this_date < end_date:  # Coupon payment dates
            this_date = this_date + delta
            if this_date < modelling_date: #Not interested in past payments
                continue
            if this_date <= end_date:
                yield this_date # ? What is the advantage of yield here?

GetEquityShare generator imports the portfolio of equities from the csv file and correctly saves them in the EquityShare objects.

In [52]:
def GetEquityShare(filename:str) -> EquityShare:
    """
    :type filename: str
    """

    with open(filename, mode="r", encoding="utf-8-sig") as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            equity_share = EquityShare(asset_id=int(row["Asset_ID"]),
                             nace=row["NACE"],
                             issuer=None,
                             issue_date=datetime.strptime(row["Issue_Date"], "%d/%m/%Y").date(),
                             dividend_yield=float(row["Dividend_Yield"]),
                             frequency=int(row["Frequency"]),
                             market_price=float(row["Market_Price"]),
                             growth_rate= float(row["Growth_Rate"]))
            yield equity_share
            

In [53]:
class EquitySharePortfolio():
    def __init__(self, equity_share: dict[int,EquityShare] = None):
        """

        :type equity_share: dict[int,EquityShare]
        """
        self.equity_share = equity_share

    def IsEmpty(self)-> bool:
        if self.equity_share == None:
            return True
        if len(self.equity_share) == 0:
            return True
        return False

    def add(self, equity_share: EquityShare) :
        """

        :type equity_share: EquityShare
        """
        if self.equity_share == None:
            self.equity_share = {equity_share.asset_id: equity_share}
        else:
            self.equity_share.update({equity_share.asset_id: equity_share})



    def create_dividend_dates(self, modelling_date: date, end_date: date)->dict:
        """
        Create the vector of dates at which the dividends are paid out and the total amounts for
        all equity shares in the portfolio, for dates on or after the modelling date.

        Parameters
        ----------
        self : EquitySharePortfolio class instance
            The EquitySharePortfolio instance with populated initial portfolio.
        :type modelling_date: date
        :type end_date: date

        Returns
        -------
        all_dividends
            An array of arrays of datetimes, containing all the dates at which the coupons are paid out.

        """
        all_dividends = []
        dividends: dict[date, float] = {}
        equity_share: EquityShare
        dividend_date: date
        for asset_id in self.equity_share:
            equity_share = self.equity_share[asset_id] # Select one asset position
            dividend_amount = 0
            for dividend_date in equity_share.generate_dividend_dates(modelling_date, end_date):
                if dividend_date in dividends: # If two cash flows on same date
                    pass
                    # Do nothing since dividend amounts are calibrated afterwards for equity
                    #dividends[dividend_date] = dividend_amount + dividends[dividend_date] # ? Why is here a plus? (you agregate coupon amounts if same date?)
                else: # New cash flow date
                    dividends.update({dividend_date:dividend_amount})
            all_dividends.append(dividends)
        return all_dividends

    def create_terminal_dates(self, terminal_date: date) -> dict:
        """
        self : EquitySharePortfolio class instance
            The EquitySharePortfolio instance with populated initial portfolio.
        :type modelling_date: date
        :type end_date: date

        :rtype: dict
        """
        all_terminals = []
        terminals: dict[date, float] = {}
        equity_share: EquityShare
        terminal_date: date

        for asset_id in self.equity_share:
            equity_share = self.equity_share[asset_id]
            
            terminal_amount = 0
            terminal_date = end_date
            if terminal_date in terminals:
                pass
                # Do nothing since dividend amounts are calibrated afterwards for equity
                #terminals[terminal_date] = terminal_amount + terminals[terminal_date]
            else:
                terminals.update({terminal_date:terminal_amount})
            all_terminals.append(terminals)
        return all_terminals


In [54]:
# Check equity construction
asset_id = 1
nace = "A.1.2"
issuer = "Open Source Modelling"
issue_date = date(2015, 12, 1)
dividend_yield = 0.03
frequency = Frequency.QUARTERLY
market_price = 12.6
growth_rate = 0.02

test_share_1 = EquityShare(asset_id = asset_id, nace= nace,
    issuer= issuer
    ,issue_date= issue_date
    ,dividend_yield= dividend_yield
    ,frequency= frequency
    ,market_price= market_price
    ,growth_rate = growth_rate
)

assert test_share_1.asset_id == asset_id
assert test_share_1.nace == nace
assert test_share_1.issuer == issuer
assert test_share_1.issue_date == issue_date
assert test_share_1.dividend_yield == dividend_yield
assert test_share_1.frequency == frequency
assert test_share_1.market_price == market_price
assert test_share_1.growth_rate == growth_rate

In [55]:
test_share_2 = EquityShare(asset_id= 2, nace= "A.3.1",
    issuer= "Test Issuer"
    ,issue_date= date(2016, 7, 1)
    ,dividend_yield= 0.04
    ,frequency= Frequency.MONTHLY
    ,market_price= 102.1
    ,growth_rate = 0.04
)

In [56]:
test_share_3 = EquityShare(asset_id= 3, nace= "B.12",
    issuer= "Test Issuer"
    ,issue_date= date(2020, 5, 2)
    ,dividend_yield= 0.012
    ,frequency= Frequency.TRIANNUAL
    ,market_price= 102
    ,growth_rate = 0.01
)

In [57]:
# Test is Portfolio constructor
a = EquitySharePortfolio()

test_equity_import = GetEquityShare("Input\Equity_Portfolio_test.csv")

#test_Not_IsEmpty
b = EquitySharePortfolio({test_share_1.asset_id: test_share_1}) 

In [58]:
# test_add_to_empty_portfolio
equity_share_portfolio = EquitySharePortfolio()
equity_share_portfolio.add(test_share_1)

# isEmpty?
equity_share_portfolio.IsEmpty()

# Only one equity in ptf
len(equity_share_portfolio.equity_share)

# Is equity identified by asset id equal to the input equity object
equity_share_portfolio.equity_share[test_share_1.asset_id] == test_share_1

test_share_1.asset_id in equity_share_portfolio.equity_share

True

In [59]:
# test_add_to_non_empty_portfolio

equity_share_portfolio = EquitySharePortfolio()
equity_share_portfolio.add(test_share_1)
equity_share_portfolio.add(test_share_2)
equity_share_portfolio.add(test_share_3)

len(equity_share_portfolio.equity_share)

test_share_3.asset_id in equity_share_portfolio.equity_share

True

In [60]:
#test_create_dividend_dates_single_bond(test_share_1):
equity_share_portfolio = EquitySharePortfolio()
equity_share_portfolio.add(test_share_1)
modelling_date = date(2023, 6, 1)
end_date = date(2023+50, 6, 1)
dividend_dates = equity_share_portfolio.create_dividend_dates(modelling_date, end_date)
assert date(2023, 6, 1) in dividend_dates[0]
assert date(2023, 9, 1) in dividend_dates[0]
assert date(2023, 12, 1) in dividend_dates[0]
assert dividend_dates[0][date(2023, 6, 1)] == 0


In [61]:
# Calculate dividend dates for 2 shares
equity_share_portfolio = EquitySharePortfolio()
equity_share_portfolio.add(test_share_1)
equity_share_portfolio.add(test_share_2)
modelling_date = date(2023, 6, 1)
end_date = date(2023+50, 6, 1)
dividend_dates = equity_share_portfolio.create_dividend_dates(modelling_date, end_date)
equity_share_portfolio.dividend_dates = dividend_dates

assert date(2023, 6, 1) in dividend_dates[0]
assert dividend_dates[0][date(2023, 6, 1)] == 0
assert date(2023, 7, 1) in dividend_dates[0]
assert date(2023, 7, 1) in dividend_dates[1]

In [67]:
# Calculate dividend dates for 2 shares
equity_share_portfolio = EquitySharePortfolio()
equity_share_portfolio.add(test_share_1)
equity_share_portfolio.add(test_share_2)
modelling_date = date(2023, 6, 1)
end_date = date(2023+50, 6, 1)
terminal_dates = equity_share_portfolio.create_terminal_dates(end_date)
equity_share_portfolio.terminal_dates = terminal_dates


assert date(2023, 6, 1) in dividend_dates[0]
assert dividend_dates[0][date(2023, 6, 1)] == 0
assert date(2023, 7, 1) in dividend_dates[0]
assert date(2023, 7, 1) in dividend_dates[1]
assert end_date in terminal_dates[0]
assert end_date in terminal_dates[1]

In [66]:
terminal_dates[0]

{datetime.date(2073, 6, 1): 0}

In [62]:
def create_dividend_fractions(obj, modelling_date)->dict:
    """
    Create the vector of year fractions at which the dividends are paid out and the total amounts for
    all equity shares in the portfolio, for dates on or after the modelling date

    Parameters
    ----------
    self : EquitySharePortfolio class instance
        The EquitySharePortfolio instance with populated initial portfolio.

    Returns
    -------
    EquityShare.dividendfrac
        An array of flats, containing all the date fractions at which the dividends are paid out.

    """        

    # other counting conventions MISSING

    n_equity = len(obj.equity_share)  # Number of assets in the bond portfolio

    # Data structures list of lists for dividend payments
    all_date_frac = (
        []
    )  # this will save the date fractions of dividends for the portfolio
    all_dates_considered = (
        []
    )  # this will save if a cash flow is already expired before the modelling date in the portfolio

    # Data structure list of lists for terminal amount repayment
    all_dividend_date_frac = (
        []
    )  # this will save the date fractions of terminal value repayment for the portfolio
    all_dividend_dates_considered = (
        []
    )  # this will save if a bond is already expired before the modelling date in the portfolio

    for i_equity in range(0, n_equity):  # For each equity in the current portfolio
        # Reset objects for the next asset
        equity_date_frac = np.array(
            []
        )  # this will save date fractions of dividends of a single asset
        equity_dates_considered = np.array(
            []
        )  # this will save the boolean, if the dividend date is after the modelling date

        equity_terminal_date_frac = np.array(
            []
        ) # this will save date fractions of terminal sale of a single equity
        equity_terminal_dates_considered = np.array(
            []
        )  # this will save the boolean, if the terminal sale date is after the modelling date

        dividend_counter = 0  # Counter of future dividend cash flows initialized to 0

        for one_dividend_date in obj.dividend_dates[i_equity]:  # for each coupon date of the selected bond
            dividend_days = (one_dividend_date - modelling_date).days
            if dividend_days > 0:  # coupon date is after the modelling date
                equity_date_frac = np.append(
                    equity_date_frac, dividend_days / 365.25
                )  # append date fraction
                equity_dates_considered = np.append(
                    equity_dates_considered, int(dividend_counter)
                )  # append "is after modelling date" flag
            dividend_counter += 1
            # else skip
        all_date_frac.append(
            equity_date_frac
        )  # append what fraction of the date is each cash flow compared to the modelling date
        all_dates_considered.append(
            equity_dates_considered.astype(int)
        )  # append which cash flows are after the modelling date

        # Calculate if the terminal sale date is before the modelling date

        
         #MISSING DEFINE TERMINAL DATES
        asset_calc_terminal_date_frac = list(obj.terminal_dates[i_equity])[0] - modelling_date
          # calculate the time difference between the terminal sale date and modelling date
        #terminal_days
        if (
            asset_calc_terminal_date_frac.days > 0
        ):  # if terminal sale date is after modelling date
            equity_terminal_date_frac = np.append(
                equity_terminal_date_frac, asset_calc_terminal_date_frac.days / 365.25
            )  # append date fraction
            equity_terminal_dates_considered = np.append(
                equity_terminal_dates_considered, int(1)
            )  # append "is after modelling date" flag
        # else skip
        all_dividend_date_frac.append(
            equity_terminal_date_frac
        )  # append what fraction of the date is each cash flow compared to the modelling date
        all_dividend_dates_considered.append(
            equity_terminal_dates_considered.astype(int)
        )  # append which cash flows are after the modelling date

    # Save coupon related data structures into the object
    obj.dividend_dates_frac = all_date_frac
    obj.dates_considered = all_dates_considered

    # Save terminal sale related data structures into the object
    obj.terminal_dates_frac = all_dividend_date_frac
    obj.dates_considerednot = all_dividend_dates_considered

    return [
        all_date_frac,
        all_dates_considered,
        all_dividend_date_frac,
        all_dividend_dates_considered,
    ]  # return all generated data structures (for now)




In [63]:
create_dividend_fractions(equity_share_portfolio, modelling_date)

[[array([ 0.25188227,  0.50102669,  0.75017112,  1.00205339,  1.25393566,
          1.50308008,  1.74948665,  2.00136893,  2.2532512 ,  2.50239562,
          2.74880219,  3.00068446,  3.25256674,  3.50171116,  3.74811773,
          4.        ,  4.25188227,  4.50102669,  4.75017112,  5.00205339,
          5.25393566,  5.50308008,  5.74948665,  6.00136893,  6.2532512 ,
          6.50239562,  6.74880219,  7.00068446,  7.25256674,  7.50171116,
          7.74811773,  8.        ,  8.25188227,  8.50102669,  8.75017112,
          9.00205339,  9.25393566,  9.50308008,  9.74948665, 10.00136893,
         10.2532512 , 10.50239562, 10.74880219, 11.00068446, 11.25256674,
         11.50171116, 11.74811773, 12.        , 12.25188227, 12.50102669,
         12.75017112, 13.00205339, 13.25393566, 13.50308008, 13.74948665,
         14.00136893, 14.2532512 , 14.50239562, 14.74880219, 15.00068446,
         15.25256674, 15.50171116, 15.74811773, 16.        , 16.25188227,
         16.50102669, 16.75017112, 17.

Question 1: Why did you put all date fractions in a single dimension list?
Question 2: Is it worth using numpy arrays so that we can do vector substractions?